# 🧠 InsightSpike-AI Google Colab Demo

**Brain-Inspired Multi-Agent Architecture for Insight Detection**

このノートブックは、InsightSpike-AIをGoogle Colab環境で実行するためのデモです。

⚡ **GPU Runtime推奨**: Runtime > Change runtime type > GPU

## 🕒 Setup Time Estimates
- **Fast Setup**: ~3-5 分 (PyTorch Geometric なし)
- **Standard Setup**: ~8-12 分 (完全インストール)
- **Debug Setup**: ~15-20 分 (詳細ログ付き)

## 🚀 Setup Options
3つのセットアップオプションから選択してください：

## 🔧 セットアップ

In [ ]:
# リポジトリのクローン
!git clone https://github.com/miyauchikazuyoshi/InsightSpike-AI.git
%cd InsightSpike-AI

In [ ]:
# 🚀 セットアップオプション選択メニュー
# このセルを先に実行して、どのセットアップを使うか決めてください

from IPython.display import HTML, display

html_menu = """
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; font-family: Arial, sans-serif;">
    <h2 style="text-align: center; margin-bottom: 20px;">🧠 InsightSpike-AI セットアップ選択</h2>
    
    <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px; margin-bottom: 20px;">
        
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #4CAF50;">
            <h3 style="margin: 0 0 10px 0; color: #4CAF50;">⚡ 1. Fast Setup (推奨)</h3>
            <p style="margin: 5px 0;"><strong>時間:</strong> 3-5分</p>
            <p style="margin: 5px 0;"><strong>用途:</strong> 開発・テスト</p>
            <p style="margin: 5px 0;"><strong>特徴:</strong> タイムアウト保護付き</p>
            <p style="margin: 5px 0; font-size: 0.9em;">PyTorch Geometric: ✅ 保護付き</p>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #2196F3;">
            <h3 style="margin: 0 0 10px 0; color: #2196F3;">🚀 2. Minimal Setup</h3>
            <p style="margin: 5px 0;"><strong>時間:</strong> 60秒未満</p>
            <p style="margin: 5px 0;"><strong>用途:</strong> クイック検証</p>
            <p style="margin: 5px 0;"><strong>特徴:</strong> 超高速</p>
            <p style="margin: 5px 0; font-size: 0.9em;">PyTorch Geometric: ❌ なし</p>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #FF9800;">
            <h3 style="margin: 0 0 10px 0; color: #FF9800;">🔍 3. Debug Setup</h3>
            <p style="margin: 5px 0;"><strong>時間:</strong> 15-20分</p>
            <p style="margin: 5px 0;"><strong>用途:</strong> トラブルシューティング</p>
            <p style="margin: 5px 0;"><strong>特徴:</strong> 詳細ログ</p>
            <p style="margin: 5px 0; font-size: 0.9em;">PyTorch Geometric: ✅ モニタリング付き</p>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #f44336;">
            <h3 style="margin: 0 0 10px 0; color: #f44336;">🔧 4. Full Setup</h3>
            <p style="margin: 5px 0;"><strong>時間:</strong> 8-15分</p>
            <p style="margin: 5px 0;"><strong>用途:</strong> 完全機能</p>
            <p style="margin: 5px 0;"><strong>特徴:</strong> 従来版</p>
            <p style="margin: 5px 0; font-size: 0.9em;">PyTorch Geometric: ⚠️ ハングアップの可能性</p>
        </div>
        
    </div>
    
    <div style="background: rgba(255,255,255,0.15); padding: 15px; border-radius: 10px; text-align: center;">
        <h3 style="margin: 0 0 10px 0;">🎯 推奨フロー</h3>
        <p style="margin: 5px 0;"><strong>初回利用:</strong> Fast Setup (1) → テスト → 問題なければそのまま使用</p>
        <p style="margin: 5px 0;"><strong>インストール問題:</strong> Debug Setup (3) → ログを確認 → 問題特定</p>
        <p style="margin: 5px 0;"><strong>急いでいる:</strong> Minimal Setup (2) → 基本機能のみテスト</p>
    </div>
    
</div>
"""

display(HTML(html_menu))

print("\n📝 使い方:")
print("1. 上のメニューで選択を決める")
print("2. リポジトリをクローンする")
print("3. セットアップセルで SETUP_CHOICE を編集して実行")
print("4. 検証セルでインストールを確認")

In [ ]:
# 環境セットアップ（GPU対応）
# 必要なスクリプトファイルに実行権限を付与
!chmod +x scripts/colab/setup_colab.sh
!chmod +x scripts/setup/setup.sh
!chmod +x scripts/setup/refactor_prepare.sh

# Pythonスクリプトにも実行権限を付与（必要に応じて）
!find scripts/ -name "*.py" -exec chmod +x {} \;

# Colabセットアップスクリプト実行
# 🎯 インタラクティブセットアップ選択
# 下のセルを実行して、適切なセットアップオプションを選択してください

print("🚀 InsightSpike-AI セットアップオプション選択")
print("="*50)
print("1️⃣ Fast Setup (推奨) - 3-5分")
print("   ✅ 開発・テスト用")
print("   ⚠️ PyTorch Geometric: タイムアウト保護付き")
print("   ✅ 基本機能: 完全対応")
print()
print("2️⃣ Minimal Setup (超高速) - 60秒未満")
print("   ✅ クイック検証用")
print("   ❌ PyTorch Geometric: なし")
print("   ✅ 基本機能: 対応")
print()
print("3️⃣ Debug Setup (トラブルシューティング) - 15-20分")
print("   🔍 問題の詳細分析")
print("   📊 詳細ログ出力")
print("   ✅ 完全機能")
print()
print("4️⃣ Full Setup (従来版) - 8-15分")
print("   ⚠️ ハングアップの可能性")
print("   ✅ 完全機能（安定時）")
print()
print("👇 下のセルで選択してください")

In [ ]:
# ✨ セットアップ選択 & 実行
# 👆 上のメニューで選択を決めたら、下の SETUP_CHOICE を編集してください

# ================================================
# ここを編集: 1, 2, 3, または 4 を入力
SETUP_CHOICE = 1  # ← この数字を変更！
# ================================================

import os
import subprocess
import time
from IPython.display import display, HTML, clear_output

# セットアップオプション定義
setup_commands = {
    1: {
        'name': '⚡ Fast Setup',
        'script': 'scripts/colab/setup_colab_fast.sh',
        'description': '推奨: 開発・テスト用',
        'time': '3-5分',
        'color': '#4CAF50'
    },
    2: {
        'name': '🚀 Minimal Setup', 
        'script': 'scripts/colab/setup_colab_minimal.sh',
        'description': '超高速: 基本機能のみ',
        'time': '60秒未満',
        'color': '#2196F3'
    },
    3: {
        'name': '🔍 Debug Setup',
        'script': 'scripts/colab/setup_colab_debug.sh', 
        'description': 'トラブルシューティング用',
        'time': '15-20分',
        'color': '#FF9800'
    },
    4: {
        'name': '🔧 Full Setup',
        'script': 'scripts/colab/setup_colab.sh',
        'description': '従来の完全セットアップ',
        'time': '8-15分',
        'color': '#f44336'
    }
}

def display_progress_bar(step, total_steps, description):
    """リアルタイム進捗バーを表示"""
    percentage = (step / total_steps) * 100
    bar_length = 40
    filled_length = int(bar_length * step // total_steps)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    progress_html = f"""
    <div style="background: #f0f0f0; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h4 style="margin: 0 0 10px 0;">🔄 セットアップ進行状況</h4>
        <div style="background: #ddd; border-radius: 10px; overflow: hidden;">
            <div style="background: linear-gradient(90deg, #4CAF50, #45a049); width: {percentage:.1f}%; height: 25px; transition: width 0.3s ease;"></div>
        </div>
        <p style="margin: 10px 0 0 0; text-align: center;">{bar} {percentage:.1f}%</p>
        <p style="margin: 5px 0 0 0; text-align: center; font-style: italic;">{description}</p>
    </div>
    """
    
    clear_output(wait=True)
    display(HTML(progress_html))

if SETUP_CHOICE in setup_commands:
    selected = setup_commands[SETUP_CHOICE]
    
    # 選択されたオプションを表示
    selection_html = f"""
    <div style="background: {selected['color']}; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
        <h3 style="margin: 0;">🎯 選択されたセットアップ: {selected['name']}</h3>
        <p style="margin: 5px 0;">{selected['description']} (予想時間: {selected['time']})</p>
    </div>
    """
    
    display(HTML(selection_html))
    
    # セットアップ前の準備
    display_progress_bar(1, 5, "準備中: 実行権限の設定...")
    time.sleep(0.5)
    
    # 実行権限を付与
    os.system(f"chmod +x {selected['script']}")
    
    display_progress_bar(2, 5, "実行開始: セットアップスクリプト起動...")
    time.sleep(0.5)
    
    print(f"🚀 実行中: {selected['script']}")
    print("="*60)
    print(f"\n🔋 {selected['name']} 開始...")
    print(f"📅 開始時刻: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"⏱️ 予想完了時間: {selected['time']}")
    print("\n📋 セットアップログ:")
    print("-"*40)
    
    # セットアップ実行
    start_time = time.time()
    result = os.system(f"./{selected['script']}")
    end_time = time.time()
    
    elapsed_time = end_time - start_time
    minutes = int(elapsed_time // 60)
    seconds = int(elapsed_time % 60)
    
    display_progress_bar(5, 5, "完了: セットアップ検証中...")
    
    if result == 0:
        success_html = f"""
        <div style="background: #4CAF50; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
            <h3 style="margin: 0;">✅ {selected['name']} 完了!</h3>
            <p style="margin: 5px 0;">実行時間: {minutes}分{seconds}秒</p>
            <p style="margin: 5px 0;">🎉 セットアップが正常に完了しました</p>
        </div>
        """
        display(HTML(success_html))
    else:
        error_html = f"""
        <div style="background: #f44336; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
            <h3 style="margin: 0;">❌ セットアップエラー</h3>
            <p style="margin: 5px 0;">実行時間: {minutes}分{seconds}秒</p>
            <p style="margin: 5px 0;">🔍 Debug Setup (3) で詳細確認を推奨</p>
        </div>
        """
        display(HTML(error_html))
    
    print(f"\n📊 セットアップ完了 - 実行時間: {minutes}分{seconds}秒")
    print("\n🔍 次のステップ: 下の検証セルでインストール状況を確認してください")
    
else:
    error_html = f"""
    <div style="background: #f44336; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
        <h3 style="margin: 0;">❌ 無効な選択: {SETUP_CHOICE}</h3>
        <p style="margin: 5px 0;">有効な選択: 1 (Fast), 2 (Minimal), 3 (Debug), 4 (Full)</p>
    </div>
    """
    
    display(HTML(error_html))
    print("👆 SETUP_CHOICE を 1, 2, 3, または 4 に変更して再実行してください")

In [ ]:
# ## 📋 セットアップオプション詳細
# 
# 上のセルで `SETUP_CHOICE = 1` の数字を変更するだけで、簡単にセットアップオプションを選択できます。
# 
# ### 🎯 おすすめの選択
# - **初回利用**: `SETUP_CHOICE = 1` (Fast Setup)
# - **とりあえず試したい**: `SETUP_CHOICE = 2` (Minimal Setup)  
# - **インストールで問題が発生**: `SETUP_CHOICE = 3` (Debug Setup)
# - **全機能が必要**: `SETUP_CHOICE = 4` (Full Setup)
# 
# ### ⚠️ 注意事項
# - Colab の GPU ランタイムを必ず有効にしてください
# - セットアップ中に中断された場合は、ランタイムをリスタートしてから再実行してください

In [ ]:
# 包括的環境検証の実行
# setup_colab.shで自動実行されていましたが、新しいインタラクティブシステムに合わせて手動で実行する必要があります
!python scripts/colab/test_colab_env.py

## 📊 セットアップ状況モニタリング

セットアップが完了したら、下の検証セルを実行して正常にインストールされているか確認してください。

### 🚫 エラーが発生した場合
1. **タイムアウトエラー**: ランタイムをリスタートして Minimal Setup (2) を試してください
2. **GPU関連エラー**: Runtime > Change runtime type > GPU を確認
3. **モジュールエラー**: Debug Setup (3) で詳細ログを確認
4. **その他**: [Colab トラブルシューティングガイド](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/guides/COLAB_TROUBLESHOOTING_GUIDE.md) を参照

### ✅ 次のステップ
セットアップが成功したら、下の検証セルでインストール状況を確認してから、データ準備セクションに進んでください。

## 🔍 Setup Validation and Troubleshooting

In [ ]:
# Quick validation - Run this after any setup option
print("🔍 Validating InsightSpike-AI setup...")

try:
    import torch
    cuda_available = torch.cuda.is_available()
    cuda_device_count = torch.cuda.device_count() if cuda_available else 0
    torch_version = torch.__version__
    
    if cuda_available:
        gpu_name = torch.cuda.get_device_name(0)
        print(f"✅ PyTorch {torch_version} (CUDA: Available - {gpu_name}, Devices: {cuda_device_count})")
    else:
        print(f"⚠️ PyTorch {torch_version} (CUDA: Not available - GPU acceleration disabled)")
        print("🔄 Tip: Consider enabling GPU acceleration for better performance")
except ImportError:
    print("❌ PyTorch not available")

try:
    import torch_geometric
    pyg_version = torch_geometric.__version__
    print(f"✅ PyTorch Geometric {pyg_version} (Graph neural network support)")
except ImportError:
    print("⚠️ PyTorch Geometric not available (OK for fast/minimal setup)")

try:
    import faiss
    faiss_version = faiss.__version__
    gpu_count = faiss.get_num_gpus() if hasattr(faiss, 'get_num_gpus') else 0
    
    if gpu_count > 0:
        print(f"✅ FAISS {faiss_version} (GPU acceleration: Available - {gpu_count} GPUs)")
    else:
        print(f"⚠️ FAISS {faiss_version} (GPU acceleration not available - using CPU version)")
except ImportError:
    print("❌ FAISS not available")

try:
    import transformers, datasets
    transformers_version = transformers.__version__
    print(f"✅ HuggingFace Transformers {transformers_version} (Language models and datasets ready)")
except ImportError:
    print("❌ HuggingFace libraries not available")

try:
    import sys
    sys.path.insert(0, 'src')
    from insightspike.core.config import get_config
    print("✅ InsightSpike-AI Core ready")
except ImportError as e:
    print(f"⚠️ InsightSpike-AI Core not ready - {str(e)}")

In [ ]:
# FAISS-GPU動作確認（詳細版）
# 🚀 FAISS GPU パフォーマンスベンチマーク & 診断
# GPUアクセラレーションの動作確認とパフォーマンス測定

import faiss
import numpy as np
import time
from IPython.display import display, HTML

def create_performance_card(title, cpu_time, gpu_time=None, details=""):
    """Performance comparison card generator"""
    if gpu_time is not None:
        speedup = cpu_time / gpu_time if gpu_time > 0 else 0
        speedup_color = "#4CAF50" if speedup > 2 else "#FF9800" if speedup > 1 else "#f44336"
        speedup_text = f"{speedup:.1f}x 高速化" if speedup > 1 else f"{speedup:.1f}x (遅い)"
        
        return f"""
        <div style="background: white; border: 1px solid #ddd; border-radius: 8px; padding: 15px; margin: 10px 0; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <h4 style="margin: 0 0 10px 0; color: #333;">📋 {title}</h4>
            <div style="display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 10px; text-align: center;">
                <div style="background: #f5f5f5; padding: 10px; border-radius: 6px;">
                    <div style="font-size: 0.8em; color: #666;">CPU</div>
                    <div style="font-weight: bold; color: #333;">{cpu_time:.4f}s</div>
                </div>
                <div style="background: #e8f5e8; padding: 10px; border-radius: 6px;">
                    <div style="font-size: 0.8em; color: #666;">GPU</div>
                    <div style="font-weight: bold; color: #333;">{gpu_time:.4f}s</div>
                </div>
                <div style="background: {speedup_color}; color: white; padding: 10px; border-radius: 6px;">
                    <div style="font-size: 0.8em;">Speedup</div>
                    <div style="font-weight: bold;">{speedup_text}</div>
                </div>
            </div>
            <p style="margin: 10px 0 0 0; font-size: 0.9em; color: #666;">{details}</p>
        </div>
        """
    else:
        return f"""
        <div style="background: #fff3cd; border: 1px solid #ffeaa7; border-radius: 8px; padding: 15px; margin: 10px 0;">
            <h4 style="margin: 0 0 10px 0; color: #856404;">⚠️ {title}</h4>
            <p style="margin: 0; color: #856404;">CPU時間: {cpu_time:.4f}s | {details}</p>
        </div>
        """

print("🔍 FAISS GPU パフォーマンステスト開始...")
print("="*60)

# FAISSバージョン情報
print(f"📚 FAISS Version: {faiss.__version__}")
print(f"🎨 GPU Resources: {hasattr(faiss, 'StandardGpuResources')}")

performance_cards = []

# GPUリソース検証
if hasattr(faiss, 'StandardGpuResources'):
    try:
        # テストパラメータ設定
        dimensions = [128, 256, 512]  # 異なる次元でテスト
        vector_counts = [1000, 10000]  # 異なるベクトル数でテスト
        
        print(f"\n🧪 テストパラメータ:")
        print(f"  - 次元: {dimensions}")
        print(f"  - ベクトル数: {vector_counts}")
        print(f"  - 検索クエリ: 10件")
        
        # GPUリソース初期化
        gpu_res = faiss.StandardGpuResources()
        print(f"\n🔋 GPUリソース初期化成功")
        
        # 異なる設定でベンチマーク
        for dim in dimensions:
            for n_vec in vector_counts:
                print(f"\n📋 テスト: {dim}D, {n_vec:,}ベクトル")
                
                # テストデータ生成
                vectors = np.random.random((n_vec, dim)).astype('float32')
                query = np.random.random((1, dim)).astype('float32')
                
                # CPUインデックス
                index_cpu = faiss.IndexFlatL2(dim)
                index_cpu.add(vectors)
                
                # GPUインデックス
                try:
                    gpu_index = faiss.index_cpu_to_gpu(gpu_res, 0, index_cpu)
                    
                    # CPU検索ベンチマーク
                    start = time.time()
                    cpu_distances, cpu_indices = index_cpu.search(query, 10)
                    cpu_time = time.time() - start
                    
                    # GPU検索ベンチマーク
                    start = time.time()
                    gpu_distances, gpu_indices = gpu_index.search(query, 10)
                    gpu_time = time.time() - start
                    
                    # 結果の一致性確認
                    accuracy = np.sum(cpu_indices == gpu_indices) / len(cpu_indices.flatten()) * 100
                    
                    performance_cards.append(create_performance_card(
                        f"{dim}D x {n_vec:,}ベクトル検索",
                        cpu_time, gpu_time,
                        f"精度: {accuracy:.1f}% | メモリ: {vectors.nbytes/1024/1024:.1f}MB"
                    ))
                    
                except Exception as e:
                    performance_cards.append(create_performance_card(
                        f"{dim}D x {n_vec:,}ベクトル検索",
                        0, None,
                        f"GPUエラー: {str(e)[:50]}..."
                    ))
        
        # メモリ使用量テスト
        print(f"\n💾 GPUメモリ使用量テスト...")
        try:
            # 大きなデータセットでメモリ測定
            large_vectors = np.random.random((50000, 256)).astype('float32')
            large_index = faiss.IndexFlatL2(256)
            large_index.add(large_vectors)
            
            large_gpu_index = faiss.index_cpu_to_gpu(gpu_res, 0, large_index)
            memory_mb = large_vectors.nbytes / 1024 / 1024
            
            performance_cards.append(create_performance_card(
                f"💾 GPUメモリテスト",
                0, 0,
                f"成功: {memory_mb:.1f}MBのデータをGPUにロード"
            ))
            
        except Exception as e:
            performance_cards.append(create_performance_card(
                f"💾 GPUメモリテスト",
                0, None,
                f"メモリエラー: {str(e)[:50]}..."
            ))
        
    except Exception as e:
        performance_cards.append(create_performance_card(
            "GPUリソース初期化",
            0, None,
            f"初期化失敗: {str(e)}"
        ))
        print(f"❌ GPUテスト失敗: {e}")

else:
    performance_cards.append(create_performance_card(
        "GPUサポート",
        0, None,
        "GPUリソースが利用できません - CPU版FAISSを使用"
    ))

# 結果表示
results_html = f"""
<div style="background: #f8f9fa; padding: 20px; border-radius: 15px; margin: 20px 0;">
    <h2 style="text-align: center; margin-bottom: 20px; color: #333;">📋 FAISS GPU パフォーマンスレポート</h2>
    {''.join(performance_cards)}
</div>
"""

display(HTML(results_html))

# パフォーマンスサマリ
if hasattr(faiss, 'StandardGpuResources'):
    print(f"\n📋 パフォーマンスサマリ:")
    print(f"  ✅ GPUリソース: 利用可能")
    print(f"  📊 テスト完了: {len(performance_cards)}件")
    print(f"  🚀 推奨: 中程度サイズのデータセットではGPUが有効")
else:
    print(f"\n📋 パフォーマンスサマリ:")
    print(f"  ⚠️ GPUリソース: 利用不可")
    print(f"  💻 CPUモード: 小規模データセットに適合")
    print(f"  💡 推奨: Runtime > Change runtime type > GPU でGPUを有効化")

print(f"\n🕰️ テスト完了時刻: {time.strftime('%Y-%m-%d %H:%M:%S')}")

## 📊 データ準備

In [ ]:
# CLIコマンドアクセス確認
print("\n💻 CLI Command Access Validation:")

# Poetry CLIアクセステスト
try:
    import subprocess
    result = subprocess.run(['poetry', 'run', 'python', '-c', 'import insightspike.cli; print("CLI module accessible")'], 
                          capture_output=True, text=True, cwd='.')
    if result.returncode == 0:
        print("✅ Poetry CLI access: Available")
        print(f"   Output: {result.stdout.strip()}")
    else:
        print(f"❌ Poetry CLI access failed: {result.stderr}")
except Exception as e:
    print(f"❌ Poetry CLI test error: {e}")

# 直接アクセステスト
try:
    import sys
    import os
    old_path = sys.path.copy()
    if 'src' not in sys.path:
        sys.path.insert(0, 'src')
    
    import insightspike.cli
    print("✅ Direct CLI module access: Available")
    
    # PYTHONPATHテスト
    os.environ['PYTHONPATH'] = 'src'
    result = subprocess.run(['python', '-c', 'import insightspike.cli; print("Direct access works")'], 
                          capture_output=True, text=True, env=os.environ)
    if result.returncode == 0:
        print("✅ PYTHONPATH direct access: Available")
    else:
        print(f"⚠️ PYTHONPATH access issue: {result.stderr}")
    
    sys.path = old_path
except Exception as e:
    print(f"❌ Direct CLI access error: {e}")

print("\n🚀 Recommended CLI usage:")
print("   🎆 Primary:   !PYTHONPATH=src poetry run python -m insightspike.cli [command]")
print("   🔄 Fallback:  !PYTHONPATH=src python -m insightspike.cli [command]")

In [ ]:
# データの準備（サンプルデータ作成）
# 必要なディレクトリを作成
!mkdir -p data/raw data/processed data/embedding data/models

# サンプルデータファイルを作成
with open('data/raw/test_sentences.txt', 'w', encoding='utf-8') as f:
    f.write("""The aurora borealis is caused by charged particles from the sun interacting with Earth's magnetic field.
Quantum entanglement is a phenomenon where particles become correlated in ways that defy classical physics.
Artificial intelligence uses machine learning algorithms to process data and make predictions.
The human brain contains billions of neurons that communicate through synapses.
Photosynthesis converts sunlight into chemical energy in plants.
DNA contains the genetic instructions for all living organisms.
Gravity is a fundamental force that attracts objects with mass toward each other.
Evolution explains how species change over time through natural selection.
""")

print("✅ Data directories created")
print("✅ Sample data created in data/raw/test_sentences.txt")

In [ ]:
# エピソード記憶の構築
# Method 1: Poetry run (推奨)
!PYTHONPATH=src poetry run python -m insightspike.cli embed --path data/raw/test_sentences.txt

# Method 2: 直接実行 (Poetry問題時の代替)
# !PYTHONPATH=src python -m insightspike.cli embed --path data/raw/test_sentences.txt

# Method 3: Makefileを使用
# !make embed

In [ ]:
# 類似性グラフの構築
# Method 1: Poetry run (推奨)
!PYTHONPATH=src poetry run python -m insightspike.cli graph

# Method 2: 直接実行 (Poetry問題時の代替)
# !PYTHONPATH=src python -m insightspike.cli graph

## 🚀 デモ実行

In [ ]:
# 簡単なテスト実行
# Method 1: Poetry run (推奨)
!PYTHONPATH=src poetry run python -m insightspike.cli loop "What is quantum entanglement?"

# Method 2: 直接実行 (Poetry問題時の代替)
# !PYTHONPATH=src python -m insightspike.cli loop "What is quantum entanglement?"

# 🚀 インタラクティブデモ実行
# 質問を選択して InsightSpike-AI の動作を確認

import time
from IPython.display import display, HTML, clear_output

# デモ質問のプリセット
demo_questions = {
    1: {
        'question': 'What is quantum entanglement?',
        'category': '🔬 Physics',
        'description': '量子もつれについて - 物理学の複雑な概念'
    },
    2: {
        'question': 'How does artificial intelligence work?',
        'category': '🤖 AI/Technology', 
        'description': '人工知能の仕組み - テクノロジーの説明'
    },
    3: {
        'question': 'What causes the aurora borealis?',
        'category': '🌌 Natural Phenomena',
        'description': 'オーロラの原因 - 自然現象のメカニズム'
    },
    4: {
        'question': 'How does photosynthesis work?',
        'category': '🌱 Biology',
        'description': '光合成のプロセス - 生物学の基本概念'
    },
    5: {
        'question': 'Custom Question',
        'category': '✏️ Custom',
        'description': '独自の質問を入力'
    }
}

def create_question_menu():
    """質問選択メニューを生成"""
    menu_html = """
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 15px; color: white; margin: 20px 0;">
        <h2 style="text-align: center; margin-bottom: 20px;">🧠 InsightSpike-AI デモ質問選択</h2>
        <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 15px;">
    """
    
    for q_id, q_data in demo_questions.items():
        if q_id == 5:  # Custom question
            card_html = f"""
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #FFC107;">
                <h4 style="margin: 0 0 8px 0; color: #FFC107;">{q_data['category']}</h4>
                <h3 style="margin: 0 0 8px 0;">{q_id}. {q_data['question']}</h3>
                <p style="margin: 0; font-size: 0.9em; opacity: 0.9;">{q_data['description']}</p>
            </div>
            """
        else:
            card_html = f"""
            <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; border-left: 4px solid #4CAF50;">
                <h4 style="margin: 0 0 8px 0; color: #4CAF50;">{q_data['category']}</h4>
                <h3 style="margin: 0 0 8px 0;">{q_id}. {q_data['question']}</h3>
                <p style="margin: 0; font-size: 0.9em; opacity: 0.9;">{q_data['description']}</p>
            </div>
            """
        menu_html += card_html
    
    menu_html += """
        </div>
        <div style="background: rgba(255,255,255,0.15); padding: 15px; border-radius: 10px; margin-top: 20px; text-align: center;">
            <h3 style="margin: 0 0 10px 0;">📝 使い方</h3>
            <p style="margin: 0;">下のセルで DEMO_CHOICE を編集して実行してください</p>
        </div>
    </div>
    """
    
    return menu_html

# 質問選択メニュー表示
display(HTML(create_question_menu()))

print("\n👇 下のセルで DEMO_CHOICE を選択してデモを実行してください")

In [ ]:
# ✨ デモ実行 & 結果表示
# 上のメニューで選択した質問を実行

# ================================================
# ここを編集: 1, 2, 3, 4, または 5 を入力
DEMO_CHOICE = 1  # ← この数字を変更！
CUSTOM_QUESTION = ""  # DEMO_CHOICE = 5 の場合のみ使用
# ================================================

import subprocess
import time
import json
from IPython.display import display, HTML, clear_output

# デモ質問の定義（上のセルと同じ）
demo_questions = {
    1: {
        'question': 'What is quantum entanglement?',
        'category': '🔬 Physics',
        'description': '量子もつれについて - 物理学の複雑な概念'
    },
    2: {
        'question': 'How does artificial intelligence work?',
        'category': '🤖 AI/Technology', 
        'description': '人工知能の仕組み - テクノロジーの説明'
    },
    3: {
        'question': 'What causes the aurora borealis?',
        'category': '🌌 Natural Phenomena',
        'description': 'オーロラの原因 - 自然現象のメカニズム'
    },
    4: {
        'question': 'How does photosynthesis work?',
        'category': '🌱 Biology',
        'description': '光合成のプロセス - 生物学の基本概念'
    }
}

def create_demo_status_card(status, question, category, execution_time=None, error=None):
    """デモ実行状況カードを生成"""
    if status == "running":
        return f"""
        <div style="background: #2196F3; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
            <h3 style="margin: 0 0 10px 0;">🔄 実行中...</h3>
            <p style="margin: 0;"><strong>{category}:</strong> {question}</p>
            <div style="margin: 10px 0;">
                <div style="width: 100%; background: rgba(255,255,255,0.3); border-radius: 10px; overflow: hidden;">
                    <div style="width: 50%; height: 8px; background: white; animation: pulse 1.5s infinite;"></div>
                </div>
            </div>
        </div>
        """
    elif status == "success":
        return f"""
        <div style="background: #4CAF50; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
            <h3 style="margin: 0 0 10px 0;">✅ 実行成功!</h3>
            <p style="margin: 0;"><strong>{category}:</strong> {question}</p>
            <p style="margin: 5px 0 0 0;">実行時間: {execution_time:.1f}秒</p>
        </div>
        """
    elif status == "error":
        return f"""
        <div style="background: #f44336; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
            <h3 style="margin: 0 0 10px 0;">❌ エラー発生</h3>
            <p style="margin: 0;"><strong>{category}:</strong> {question}</p>
            <p style="margin: 5px 0 0 0; font-size: 0.9em;">{error}</p>
        </div>
        """

# 選択的実行
if DEMO_CHOICE in demo_questions:
    selected_q = demo_questions[DEMO_CHOICE]
    question = selected_q['question']
    category = selected_q['category'] 
    description = selected_q['description']
elif DEMO_CHOICE == 5 and CUSTOM_QUESTION:
    question = CUSTOM_QUESTION
    category = '✏️ Custom'
    description = 'カスタム質問'
else:
    error_html = f"""
    <div style="background: #f44336; color: white; padding: 15px; border-radius: 10px; margin: 10px 0; text-align: center;">
        <h3 style="margin: 0;">❌ 無効な選択: {DEMO_CHOICE}</h3>
        <p style="margin: 5px 0 0 0;">有効な選択: 1-4 (プリセット), 5 (カスタム)</p>
        <p style="margin: 5px 0 0 0;">カスタムの場合は CUSTOM_QUESTION を設定してください</p>
    </div>
    """
    display(HTML(error_html))
    question = None

if question:
    # 実行開始通知
    display(HTML(create_demo_status_card("running", question, category)))
    
    print(f"🚀 InsightSpike-AI デモ実行開始")
    print(f"Question: {question}")
    print(f"Category: {category}")
    print("="*60)
    
    start_time = time.time()
    
    try:
        # Method 1: Poetry run (推奨)
        print(f"\n📋 方法 1: Poetry CLI で実行")
        result = subprocess.run([
            'poetry', 'run', 'python', '-m', 'insightspike.cli', 'loop', question
        ], capture_output=True, text=True, env={'PYTHONPATH': 'src'}, timeout=120)
        
        if result.returncode == 0:
            execution_time = time.time() - start_time
            clear_output(wait=True)
            display(HTML(create_demo_status_card("success", question, category, execution_time)))
            print(f"🎉 デモ実行成功! (Poetry CLI)")
            print(f"\n📝 実行結果:")
            print("-"*40)
            print(result.stdout)
            if result.stderr:
                print(f"\n⚠️ 警告メッセージ:")
                print(result.stderr)
        else:
            # Method 2: 直接実行 (代替)
            print(f"\n📋 方法 2: 直接実行 (フォールバック)")
            result2 = subprocess.run([
                'python', '-m', 'insightspike.cli', 'loop', question
            ], capture_output=True, text=True, env={'PYTHONPATH': 'src'}, timeout=120)
            
            if result2.returncode == 0:
                execution_time = time.time() - start_time
                clear_output(wait=True)
                display(HTML(create_demo_status_card("success", question, category, execution_time)))
                print(f"🎉 デモ実行成功! (直接実行)")
                print(f"\n📝 実行結果:")
                print("-"*40)
                print(result2.stdout)
                if result2.stderr:
                    print(f"\n⚠️ 警告メッセージ:")
                    print(result2.stderr)
            else:
                execution_time = time.time() - start_time
                error_msg = result2.stderr or result.stderr or "不明なエラー"
                clear_output(wait=True)
                display(HTML(create_demo_status_card("error", question, category, None, error_msg[:100])))
                print(f"❌ デモ実行エラー")
                print(f"\n📄 エラー詳細:")
                print(f"Poetry エラー: {result.stderr}")
                print(f"直接実行エラー: {result2.stderr}")
    
    except subprocess.TimeoutExpired:
        execution_time = time.time() - start_time
        clear_output(wait=True)
        display(HTML(create_demo_status_card("error", question, category, None, "タイムアウト (120秒)")))
        print(f"⏱️ タイムアウト: 実行が120秒で中断されました")
    
    except Exception as e:
        execution_time = time.time() - start_time
        clear_output(wait=True)
        display(HTML(create_demo_status_card("error", question, category, None, str(e))))
        print(f"❌ 予期しないエラー: {e}")
    
    print(f"\n🕰️ 総実行時間: {execution_time:.1f}秒")
    print(f"📊 次のステップ: 下のインタラクティブテストで詳細な結果を確認"

else:
    print("👆 上で正しい DEMO_CHOICE を設定してから再実行してください")

## 🔍 インタラクティブテスト

In [ ]:
# 🔬 高度なインタラクティブテスト & 分析
# Python環境での詳細なシステムテストと可視化

import sys
import os
import time
import json
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import numpy as np

# 環境設定
sys.path.insert(0, 'src')
os.environ['PYTHONPATH'] = 'src'

def create_analysis_card(title, status, details, metrics=None, color="#2196F3"):
    """分析結果カードを生成"""
    metrics_html = ""
    if metrics:
        metrics_html = "<div style='display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px; margin: 10px 0;'>"
        for key, value in metrics.items():
            metrics_html += f"""
            <div style='background: rgba(255,255,255,0.2); padding: 8px; border-radius: 6px; text-align: center;'>
                <div style='font-size: 0.8em; opacity: 0.8;'>{key}</div>
                <div style='font-weight: bold;'>{value}</div>
            </div>
            """
        metrics_html += "</div>"
    
    return f"""
    <div style="background: {color}; color: white; padding: 15px; border-radius: 10px; margin: 10px 0;">
        <h4 style="margin: 0 0 10px 0;">{title}</h4>
        <p style="margin: 0 0 10px 0; opacity: 0.9;">{details}</p>
        {metrics_html}
        <div style="background: rgba(255,255,255,0.1); padding: 8px; border-radius: 6px; text-align: center; margin-top: 10px;">
            <strong>状態: {status}</strong>
        </div>
    </div>
    """

print("🔬 InsightSpike-AI 高度インタラクティブテスト開始")
print("="*60)

analysis_results = []

# システム初期化テスト
print("\n🔋 Step 1: システム初期化テスト")
try:
    from insightspike.core.layers.layer2_memory_manager import L2MemoryManager
    from insightspike.core.agents.main_agent import MainAgent
    
    init_start = time.time()
    agent = MainAgent()
    init_success = agent.initialize()
    init_time = time.time() - init_start
    
    if init_success:
        memory_stats = agent.l2_memory.get_memory_stats()
        total_episodes = memory_stats.get('total_episodes', 0)
        
        analysis_results.append(create_analysis_card(
            "🧠 システム初期化",
            "成功", 
            f"エージェントが正常に初期化されました",
            {
                "初期化時間": f"{init_time:.3f}s",
                "エピソード数": f"{total_episodes}件",
                "メモリ状態": "正常",
                "GPUサポート": "有効" if hasattr(agent.l2_memory, 'gpu_accelerated') else "不明"
            },
            "#4CAF50"
        ))
        
        print(f"✅ エージェント初期化成功 ({init_time:.3f}s)")
        print(f"✅ メモリロード成功: {total_episodes}エピソード")
    else:
        analysis_results.append(create_analysis_card(
            "🧠 システム初期化",
            "失敗", 
            "エージェントの初期化に失敗しました",
            {"初期化時間": f"{init_time:.3f}s", "エラー": "初期化失敗"},
            "#f44336"
        ))
        print(f"❌ エージェント初期化失敗")
        agent = None

except Exception as e:
    analysis_results.append(create_analysis_card(
        "🧠 システム初期化",
        "エラー", 
        f"セットアップエラー: {str(e)[:50]}...",
        {"エラータイプ": type(e).__name__},
        "#f44336"
    ))
    print(f"❌ セットアップエラー: {e}")
    agent = None

# 質問処理テスト
if agent:
    print("\n🔋 Step 2: 質問処理テスト")
    
    # テスト質問セット
    test_questions = [
        "What causes the aurora borealis?",
        "How does artificial intelligence work?",
        "What is quantum entanglement?"
    ]
    
    question_results = []
    
    for i, question in enumerate(test_questions):
        print(f"\n📋 Question {i+1}: {question}")
        
        try:
            process_start = time.time()
            result = agent.process_question(question, verbose=False)
            process_time = time.time() - process_start
            
            # 結果分析
            response = result.get('response', 'No response')
            reasoning_quality = result.get('reasoning_quality', 0)
            spike_detected = result.get('spike_detected', False)
            total_cycles = result.get('total_cycles', 0)
            
            question_results.append({
                'question': question,
                'process_time': process_time,
                'reasoning_quality': reasoning_quality,
                'spike_detected': spike_detected,
                'total_cycles': total_cycles,
                'response_length': len(response)
            })
            
            # 品質評価
            quality_status = "高品質" if reasoning_quality > 0.7 else "中程度" if reasoning_quality > 0.5 else "低品質"
            quality_color = "#4CAF50" if reasoning_quality > 0.7 else "#FF9800" if reasoning_quality > 0.5 else "#f44336"
            
            analysis_results.append(create_analysis_card(
                f"💬 Question {i+1} 結果",
                quality_status,
                f"{question[:50]}...",
                {
                    "処理時間": f"{process_time:.3f}s",
                    "品質スコア": f"{reasoning_quality:.3f}",
                    "Insight検出": "有" if spike_detected else "無",
                    "サイクル数": f"{total_cycles}"
                },
                quality_color
            ))
            
            print(f"  ✅ 処理成功 ({process_time:.3f}s)")
            print(f"  📋 品質: {reasoning_quality:.3f}, Insight: {spike_detected}, サイクル: {total_cycles}")
            
        except Exception as e:
            analysis_results.append(create_analysis_card(
                f"💬 Question {i+1} 結果",
                "エラー",
                f"{question[:50]}...",
                {"エラー": str(e)[:30]},
                "#f44336"
            ))
            print(f"  ❌ 処理エラー: {e}")
    
    # パフォーマンス分析
    if question_results:
        print("\n🔋 Step 3: パフォーマンス分析")
        
        avg_time = np.mean([r['process_time'] for r in question_results])
        avg_quality = np.mean([r['reasoning_quality'] for r in question_results])
        total_spikes = sum([r['spike_detected'] for r in question_results])
        avg_cycles = np.mean([r['total_cycles'] for r in question_results])
        
        analysis_results.append(create_analysis_card(
            "📋 パフォーマンスサマリ",
            "完了",
            f"{len(question_results)}問のテストが完了しました",
            {
                "平均処理時間": f"{avg_time:.3f}s",
                "平均品質": f"{avg_quality:.3f}",
                "Insight検出数": f"{total_spikes}/{len(question_results)}",
                "平均サイクル": f"{avg_cycles:.1f}"
            },
            "#9C27B0"
        ))

# 結果表示
results_html = f"""
<div style="background: #f8f9fa; padding: 20px; border-radius: 15px; margin: 20px 0;">
    <h2 style="text-align: center; margin-bottom: 20px; color: #333;">📋 高度インタラクティブテスト結果</h2>
    {''.join(analysis_results)}
</div>
"""

display(HTML(results_html))

# 可視化
if agent and 'question_results' in locals() and question_results:
    print("\n📋 パフォーマンス可視化")
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle('InsightSpike-AI パフォーマンス分析', fontsize=14)
    
    # 処理時間
    process_times = [r['process_time'] for r in question_results]
    ax1.bar(range(len(process_times)), process_times, color='skyblue')
    ax1.set_title('処理時間 (s)')
    ax1.set_xlabel('質問')
    ax1.set_ylabel('時間 (s)')
    
    # 品質スコア
    quality_scores = [r['reasoning_quality'] for r in question_results]
    ax2.bar(range(len(quality_scores)), quality_scores, color='lightgreen')
    ax2.set_title('推論品質スコア')
    ax2.set_xlabel('質問')
    ax2.set_ylabel('品質スコア')
    ax2.set_ylim(0, 1)
    
    # サイクル数
    cycles = [r['total_cycles'] for r in question_results]
    ax3.bar(range(len(cycles)), cycles, color='lightcoral')
    ax3.set_title('推論サイクル数')
    ax3.set_xlabel('質問')
    ax3.set_ylabel('サイクル数')
    
    # Insight検出
    spike_counts = [1 if r['spike_detected'] else 0 for r in question_results]
    ax4.pie([sum(spike_counts), len(spike_counts) - sum(spike_counts)], 
            labels=['Insight検出', 'Insight無し'], 
            autopct='%1.1f%%', colors=['gold', 'lightgray'])
    ax4.set_title('Insight検出率')
    
    plt.tight_layout()
    plt.show()

print(f"\n🎉 高度インタラクティブテスト完了!")
print(f"🕰️ テスト完了時刻: {time.strftime('%Y-%m-%d %H:%M:%S')}")

## 🔧 包括的トラブルシューティングガイド

### 🚑 緊急時のクイックフィックス

**📚 セットアップ関連エラー**
```python
# 1. ランタイムリスタート
# Runtime > Restart runtime

# 2. 緊急セットアップ (60秒以内)
SETUP_CHOICE = 2  # Minimal Setup

# 3. GPU設定確認
# Runtime > Change runtime type > GPU
```

**🔋 モジュールインポートエラー**
```python
# 1. PYTHONPATH設定
import sys, os
sys.path.insert(0, 'src')
os.environ['PYTHONPATH'] = 'src'

# 2. 直接インストール
!pip install -e .

# 3. 個別パッケージインストール
!pip install torch transformers faiss-cpu
```

**🚀 CLIコマンドエラー**
```python
# Poetryエラーの場合
!PYTHONPATH=src python -m insightspike.cli --help

# モジュールエラーの場合
!PYTHONPATH=src python scripts/run_poc_simple.py "test question"
```

---

### 📊 詳細エラー診断 & 解決方法

#### 1. **🔥 CUDA/GPU関連エラー**

**症状**: `CUDA not available`, `GPUアクセラレーション無効`

**解決手順**:
1. **GPUランタイム確認**: Runtime > Change runtime type > GPU
2. **検証コマンド**:
   ```python
   import torch
   print(f"CUDA Available: {torch.cuda.is_available()}")
   print(f"GPU Count: {torch.cuda.device_count()}")
   ```
3. **フォールバック**: CPUモードでも基本機能は使用可能

#### 2. **📦 PyTorch Geometricハングアップ**

**症状**: インストールが120秒以上停止

**解決手順**:
1. **タイムアウト保護**: Fast Setup (1) を使用
2. **スキップ**: Minimal Setup (2) でPyGなしでテスト
3. **デバッグ**: Debug Setup (3) で詳細ログ確認

#### 3. **📝 データファイルエラー**

**症状**: `FileNotFoundError`, `データが見つかりません`

**解決手順**:
1. **ディレクトリ作成**:
   ```python
   !mkdir -p data/raw data/processed data/embedding
   ```
2. **サンプルデータ再作成**: データ準備セルを再実行
3. **パス確認**:
   ```python
   import os
   print(os.listdir('data/raw'))
   ```

#### 4. **🧠 メモリエラー**

**症状**: `Out of memory`, `カーネルクラッシュ`

**解決手順**:
1. **ランタイムリスタート**: Runtime > Restart runtime
2. **メモリ清理**:
   ```python
   import gc
   gc.collect()
   ```
3. **バッチサイズ縮小**: 小さなデータセットでテスト

#### 5. **🔗 ネットワークタイムアウト**

**症状**: パッケージダウンロード失敗

**解決手順**:
1. **タイムアウト延長**:
   ```python
   !pip install --timeout 300 [package_name]
   ```
2. **ミラー使用**:
   ```python
   !pip install -i https://pypi.org/simple/ [package_name]
   ```
3. **段階的インストール**: コアパッケージから優先

---

### 📊 パフォーマンス最適化

#### 🚀 **高速化のヒント**

1. **GPUアクセラレーション**:
   - T4 GPU: 基本的な高速化
   - V100 GPU: 中規模データセットに最適
   - A100 GPU: 大規模データセットに最適

2. **メモリ最適化**:
   ```python
   # バッチサイズ調整
   batch_size = 32 if torch.cuda.is_available() else 8
   
   # メモリクリア
   torch.cuda.empty_cache() if torch.cuda.is_available() else None
   ```

3. **データサイズ調整**:
   - テスト: 1,000ベクトル以下
   - 開発: 10,000ベクトル以下
   - 本格運用: 100,000ベクトル以上

---

### 📚 サポートリソース

#### 🔗 **外部リンク**

1. **プロジェクトドキュメント**:
   - [Colabトラブルシューティングガイド](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/guides/COLAB_TROUBLESHOOTING_GUIDE.md)
   - [環境セットアップガイド](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/guides/ENVIRONMENT_SETUP_GUIDE.md)
   - [アーキテクチャ進化ロードマップ](https://github.com/miyauchikazuyoshi/InsightSpike-AI/blob/main/documentation/ARCHITECTURE_EVOLUTION_ROADMAP.md)

2. **技術サポート**:
   - [GitHub Issues](https://github.com/miyauchikazuyoshi/InsightSpike-AI/issues)
   - [Discussions](https://github.com/miyauchikazuyoshi/InsightSpike-AI/discussions)

#### 📈 **デバッグ情報収集**

**エラー報告時に含める情報**:
```python
# システム情報
import sys, torch
print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")

# エラーログ
import traceback
try:
    # エラーが発生したコード
    pass
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
```

#### 💪 **コミュニティ性能**

**結果シェア**:
- 成功事例や改善提案をGitHub Discussionsでシェア
- パフォーマンスベンチマーク結果をコミュニティに貢献

**フィードバック**:
- セットアップ時間や成功率の報告
- 新しいエラーパターンや解決方法の提案

---

### 🏁 **成功のためのチェックリスト**

✅ GPUランタイムが有効  
✅ セットアップ検証で全項目成功  
✅ FAISS GPUテストで高速化確認  
✅ デモ実行で正常な応答を取得  
✅ インタラクティブテストで品質スコア > 0.5  
✅ エラーなしで全セルが実行完了  

**✨ すべて成功したら、InsightSpike-AIが正常に動作しています！ ✨**